In [1]:
import os
import tempfile
import shutil
import pathlib

from freezegun import freeze_time

# import to register magics
import sqlmesh

In [2]:
freezer = freeze_time("2032-01-01 00:00:00")
freezer.start()

In [3]:
root_dir = pathlib.Path(os.getcwd()).parents[2]
EXAMPLE_SUSHI_DIR = pathlib.Path(root_dir) / "examples" / "sushi"
str(EXAMPLE_SUSHI_DIR)

'/home/butch2/play/study/learn-sqlmesh/sqlmesh2/examples/sushi'

In [4]:
TARGET_SUSHI_DIR = pathlib.Path(tempfile.gettempdir()) / "test_local_jupyter_notebook"
str(TARGET_SUSHI_DIR)

'/tmp/test_local_jupyter_notebook'

In [5]:
TARGET_MODEL_DIR = TARGET_SUSHI_DIR / "models"
str(TARGET_MODEL_DIR)

'/tmp/test_local_jupyter_notebook/models'

In [6]:
def ignore(src, names):
    if pathlib.Path(src).name in {".cache", "__pycache__", "logs", "data"}:
        return names
    return []

if TARGET_SUSHI_DIR.exists():
    shutil.rmtree(str(TARGET_SUSHI_DIR))
shutil.copytree(str(EXAMPLE_SUSHI_DIR), str(TARGET_SUSHI_DIR), ignore=ignore)

'/tmp/test_local_jupyter_notebook'

In [7]:
# add weird seed model
WEIRD_SEED_DATA = """Date,Start (ET),Visitor/Neutral,PTS,Home/Neutral,PTS,,,Attend.,Arena,Notes
Tue Oct 24 2023,7:30p,Los Angeles Lakers,107,Denver Nuggets,119,Box Score,,19842,Ball Arena,
Tue Oct 24 2023,10:00p,Phoenix Suns,108,Golden State Warriors,104,Box Score,,18064,Chase Center,
Wed Oct 25 2023,7:00p,Houston Rockets,86,Orlando Magic,116,Box Score,,18846,Amway Center,
"""

In [8]:
!mkdir -p {TARGET_SUSHI_DIR}/seeds
seed_data_file = TARGET_SUSHI_DIR/'seeds/weird_seed_data.csv'
with open(seed_data_file,'w') as f:
    f.write(WEIRD_SEED_DATA)

In [9]:
seed_model_file = TARGET_MODEL_DIR/'weird_seed_model.sql'
seed_model_file.write_text("""MODEL (
  name sushi.weird_seed_model,
  kind SEED (
    path '../seeds/weird_seed_data.csv'
  )
)""")

98

In [10]:
%context --config local_config $TARGET_SUSHI_DIR 

SQLMesh project context set to: /tmp/test_local_jupyter_notebook

### Leaving plan unapplied to verify that the text boxes are printed

In [11]:
%cd $TARGET_SUSHI_DIR

/tmp/test_local_jupyter_notebook


/home/butch2/play/study/learn-sqlmesh/sqlmesh2/.venv/lib/python3.11/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [12]:
# %plan

In [13]:
%plan --no-prompts --auto-apply

/home/butch2/play/study/learn-sqlmesh/sqlmesh2/sqlmesh/core/test/definition.py:183: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_datetime without passing `errors` and catch exceptions explicitly instead
  expected[col] = pd.to_datetime(expected[col], errors="ignore").dt.date  # type: ignore


HTML(value='<span style="font-size: 11px; font-weight: bold; font-family: Menlo,\'DejaVu Sans Mono\',consolas,…

New environment `prod` will be created from `prod`

Summary of differences against `prod`:

Models:
└── Added:
    ├── raw.model1
    ├── sushi.top_waiters
    ├── sushi.waiter_names
    ├── sushi.customer_revenue_by_day
    ├── sushi.weird_seed_model
    ├── sushi.orders
    ├── raw.demographics
    ├── sushi.waiter_revenue_by_day
    ├── sushi.customers
    ├── sushi.waiter_as_customer_by_day
    ├── raw.model2
    ├── sushi.marketing
    ├── sushi.customer_revenue_lifetime
    ├── sushi.order_items
    ├── sushi.waiters
    ├── sushi.items
    └── sushi.raw_marketing

...

All model versions have been created successfully

Output()

All model batches have been executed successfully

Output()

The target environment has been updated successfully

In [14]:
%plan dev --no-prompts --auto-apply --include-unmodified

/home/butch2/play/study/learn-sqlmesh/sqlmesh2/sqlmesh/core/test/definition.py:183: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_datetime without passing `errors` and catch exceptions explicitly instead
  expected[col] = pd.to_datetime(expected[col], errors="ignore").dt.date  # type: ignore


HTML(value='<span style="font-size: 11px; font-weight: bold; font-family: Menlo,\'DejaVu Sans Mono\',consolas,…

New environment `dev` will be created from `prod`

Output()

All model versions have been created successfully

Output()

The target environment has been updated successfully

Virtual Update executed successfully

In [15]:
%render sushi.top_waiters

SELECT                                                                                                             
  CAST("waiter_revenue_by_day"."waiter_id" AS INT) AS "waiter_id",                                                 
  CAST("waiter_revenue_by_day"."revenue" AS DOUBLE) AS "revenue"                                                   
FROM "local"."sqlmesh__sushi"."sushi__waiter_revenue_by_day__200393305" AS "waiter_revenue_by_day" /*              
local.sushi.waiter_revenue_by_day */                                                                               
WHERE                                                                                                              
  "waiter_revenue_by_day"."event_date" = (                                                                         
    SELECT                                                                                                         
      MAX("waiter_revenue_by_day"."event_date") AS "_col_0"                                                        
    FROM "local"."sqlmesh__sushi"."sushi__waiter_revenue_by_day__200393305" AS "waiter_revenue_by_day" /*          
local.sushi.waiter_revenue_by_day */                                                                               
  )                                                                                                                
ORDER BY                                                                                                           
  "revenue" DESC                                                                                                   
LIMIT 10                                                                                                           

In [16]:
%render sushi.top_waiters --no-format

SELECT
  CAST("waiter_revenue_by_day"."waiter_id" AS INT) AS "waiter_id",
  CAST("waiter_revenue_by_day"."revenue" AS DOUBLE) AS "revenue"
FROM "local"."sqlmesh__sushi"."sushi__waiter_revenue_by_day__200393305" AS "waiter_revenue_by_day" /* 
local.sushi.waiter_revenue_by_day */
WHERE
  "waiter_revenue_by_day"."event_date" = (
    SELECT
      MAX("waiter_revenue_by_day"."event_date") AS "_col_0"
    FROM "local"."sqlmesh__sushi"."sushi__waiter_revenue_by_day__200393305" AS "waiter_revenue_by_day" /* 
local.sushi.waiter_revenue_by_day */
  )
ORDER BY
  "revenue" DESC
LIMIT 10

In [17]:
%evaluate sushi.top_waiters

,waiter_id,revenue
0,4,317.39
1,0,273.09
2,2,250.84
3,7,155.52
4,5,124.39
5,1,60.18
6,9,48.86
7,6,21.52


In [18]:
%render sushi.weird_seed_model

SELECT                                                                                                             
  CAST(date AS TEXT) AS date,                                                                                      
  CAST(START(et) AS TEXT) AS "start (et)",                                                                         
  CAST(visitor / neutral AS TEXT) AS "visitor/neutral",                                                            
  CAST(pts AS BIGINT) AS pts,                                                                                      
  CAST(home / neutral AS TEXT) AS "home/neutral",                                                                  
  CAST(pts1 AS BIGINT) AS pts1,                                                                                    
  CAST(unnamed AS $6 AS TEXT) AS "unnamed: 6",                                                                     
  CAST(unnamed AS $7 AS DOUBLE) AS "unnamed: 7",                                                                   
  CAST(attend AS BIGINT) AS attend,                                                                                
  CAST(arena AS TEXT) AS arena,                                                                                    
  CAST(notes AS DOUBLE) AS notes                                                                                   
FROM (VALUES                                                                                                       
  ('Tue Oct 24 2023', '7:30p', 'Los Angeles Lakers', 107, 'Denver Nuggets', 119, 'Box Score', NULL, 19842, 'Ball   
Arena', NULL),                                                                                                     
  ('Tue Oct 24 2023', '10:00p', 'Phoenix Suns', 108, 'Golden State Warriors', 104, 'Box Score', NULL, 18064, 'Chase
Center', NULL),                                                                                                    
  ('Wed Oct 25 2023', '7:00p', 'Houston Rockets', 86, 'Orlando Magic', 116, 'Box Score', NULL, 18846, 'Amway       
Center', NULL)) AS t(date, "start (et)", "visitor/neutral", pts, "home/neutral", pts1, "unnamed: 6", "unnamed: 7", 
attend, arena, notes)                                                                                              

In [19]:
%evaluate sushi.weird_seed_model

,date,start (et),visitor/neutral,pts,home/neutral,pts1,unnamed: 6,unnamed: 7,attend,arena,notes
0,Tue Oct 24 2023,7:30p,Los Angeles Lakers,107,Denver Nuggets,119,Box Score,NaN,19842,Ball Arena,NaN
1,Tue Oct 24 2023,10:00p,Phoenix Suns,108,Golden State Warriors,104,Box Score,NaN,18064,Chase Center,NaN
2,Wed Oct 25 2023,7:00p,Houston Rockets,86,Orlando Magic,116,Box Score,NaN,18846,Amway Center,NaN


In [ ]:
# top_waiters = TARGET_MODEL_DIR / "top_waiters.sql"
# top_waiters.write_text("""/* View of top waiters. */
# MODEL (
#   name sushi.top_waiters,
#   owner jen,
#   audits (
#     unique_values(columns=[waiter_id])
#   ),
#   grain waiter_id
# );

# SELECT
#   waiter_id::INT AS waiter_id,
#   revenue::DOUBLE AS revenue,
#   1 as new_col
# FROM sushi.waiter_revenue_by_day
# WHERE
#   event_date = (
#     SELECT
#       MAX(event_date)
#     FROM sushi.waiter_revenue_by_day
#   )
# ORDER BY
#   revenue DESC
# LIMIT 10
# """)

In [20]:
%%model sushi.top_waiters
/* View of top waiters. */
MODEL (
  name sushi.top_waiters,
  owner jen,
  dialect duckdb,
  audits (
    UNIQUE_VALUES('columns' = (
      waiter_id
    ))
  ),
  grains (
    waiter_id
  )
);

SELECT
  waiter_id::INT AS waiter_id,
  revenue::DOUBLE AS revenue,
  1::INT AS new_col
FROM sushi.waiter_revenue_by_day
WHERE
  event_date = (
    SELECT
      MAX(event_date)
    FROM sushi.waiter_revenue_by_day
  )
ORDER BY
  revenue DESC
LIMIT 10

Model `sushi.top_waiters` updated

SELECT                                                                                                             
  CAST("waiter_revenue_by_day"."waiter_id" AS INT) AS "waiter_id",                                                 
  CAST("waiter_revenue_by_day"."revenue" AS DOUBLE) AS "revenue",                                                  
  CAST(1 AS INT) AS "new_col"                                                                                      
FROM "local"."sqlmesh__sushi"."sushi__waiter_revenue_by_day__200393305" AS "waiter_revenue_by_day" /*              
local.sushi.waiter_revenue_by_day */                                                                               
WHERE                                                                                                              
  "waiter_revenue_by_day"."event_date" = (                                                                         
    SELECT                                                                                                         
      MAX("waiter_revenue_by_day"."event_date") AS "_col_0"                                                        
    FROM "local"."sqlmesh__sushi"."sushi__waiter_revenue_by_day__200393305" AS "waiter_revenue_by_day" /*          
local.sushi.waiter_revenue_by_day */                                                                               
  )                                                                                                                
ORDER BY                                                                                                           
  "revenue" DESC                                                                                                   
LIMIT 10                                                                                                           

In [21]:
%diff prod

Summary of differences against `prod`:

Models:
└── Directly Modified:
    └── sushi.top_waiters

In [22]:
%plan dev --no-prompts --auto-apply

/home/butch2/play/study/learn-sqlmesh/sqlmesh2/sqlmesh/core/test/definition.py:183: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_datetime without passing `errors` and catch exceptions explicitly instead
  expected[col] = pd.to_datetime(expected[col], errors="ignore").dt.date  # type: ignore


HTML(value='<span style="font-size: 11px; font-weight: bold; font-family: Menlo,\'DejaVu Sans Mono\',consolas,…

Summary of differences against `dev`:

Models:
└── Directly Modified:
    └── sushi__dev.top_waiters

---                                                                                                                
                                                                                                                   
+++                                                                                                                
                                                                                                                   
@@ -9,7 +9,8 @@                                                                                                    
                                                                                                                   
 )                                                                                                                 
 SELECT                                                                                                            
   CAST(waiter_id AS INT) AS waiter_id,                                                                            
-  CAST(revenue AS DOUBLE) AS revenue                                                                              
+  CAST(revenue AS DOUBLE) AS revenue,                                                                             
+  CAST(1 AS INT) AS new_col                                                                                       
 FROM sushi.waiter_revenue_by_day                                                                                  
 WHERE                                                                                                             
   event_date = (                                                                                                  

...

Output()

All model versions have been created successfully

All model batches have been executed successfully

Output()

The target environment has been updated successfully

In [23]:
!mkdir -p $TARGET_SUSHI_DIR/html

In [24]:
%dag --file $TARGET_SUSHI_DIR/html/dag.html

In [25]:
from IPython.display import IFrame

In [26]:
IFrame(src='./dag.html', width=900, height=800)

In [27]:
%migrate

Migration complete

In [28]:
%table_diff prod:dev --model sushi.top_waiters

Schema Diff Between 'PROD' and 'DEV' environments for model 'sushi.top_waiters':
└── Added Columns:
    └── new_col (INT)

Row Counts:
├──  COMMON: 8 rows
├──  PROD ONLY: 0 rows
└──  DEV ONLY: 0 rows

COMMON ROWS column comparison stats:

pct_match
revenue      100.0

In [29]:
%run_test {TARGET_SUSHI_DIR}/tests/test_customer_revenue_by_day.yaml::test_customer_revenue_by_day


----------------------------------------------------------------------
Ran 1 test in 0.000s

OK


In [30]:
%audit sushi.top_waiters

Found 1 audit(s).

unique_values on model sushi.top_waiters ✅ PASS.

Finished with 0 audit errors and 0 audits skipped.

Done.

In [31]:
%%fetchdf my_result
SELECT 1

,1
0,1


In [32]:
my_result

,1
0,1


In [33]:
%info

Models: 17

Macros: 3

Data warehouse connection succeeded

Test connection succeeded

In [34]:
%migrate

Migration complete

In [35]:
freezer = freeze_time("2032-01-02 00:00:00")
freezer.start()

In [36]:
%run_dag

Output()

All model batches have been executed successfully

Run finished for environment 'prod'

In [37]:
# %invalidate dev

In [38]:
# %context

In [39]:
import sqlmesh

In [40]:
%create_test?

Docstring:
::

  %create_test --query QUERY [QUERY ...] [--overwrite]
                   [--var VAR [VAR ...]] [--path PATH] [--name NAME]
                   [--include-ctes]
                   model

Generate a unit test fixture for a given model.

positional arguments:
  model

options:
  --query <QUERY [QUERY ...]>, -q <QUERY [QUERY ...]>
                        Queries that will be used to generate data for the
                        model's dependencies.
  --overwrite, -o       When true, the fixture file will be overwritten in
                        case it already exists.
  --var <VAR [VAR ...]>, -v <VAR [VAR ...]>
                        Key-value pairs that will define variables needed by
                        the model.
  --path PATH, -p PATH  The file path corresponding to the fixture, relative
                        to the test directory. By default, the fixture will be
                        created under the test directory and the file name
                        w

In [ ]:
%lsmagic?

In [ ]:
%lsmagic

In [ ]:
%%model sushi.top_waiters
/* View of top waiters. */
MODEL (
  name sushi.top_waiters,
  owner jen,
  dialect duckdb,
  audits (UNIQUE_VALUES('columns' = [waiter_id])),
  grains (waiter_id)
);

SELECT
  waiter_id::INT AS waiter_id,
  revenue::DOUBLE AS revenue,
  1 AS new_col,
  2 AS new_col2
FROM sushi.waiter_revenue_by_day
WHERE
  event_date = (
    SELECT
      MAX(event_date)
    FROM sushi.waiter_revenue_by_day
  )
ORDER BY
  revenue DESC
LIMIT 100

In [ ]:
%%model sushi.top_waiters
/* View of top waiters. */
MODEL (
  name sushi.top_waiters,
  owner jen,
  dialect duckdb,
  audits (UNIQUE_VALUES('columns' = [waiter_id])),
  grains (waiter_id)
);

SELECT
  waiter_id::INT AS waiter_id,
  revenue::DOUBLE AS revenue,
  1 AS new_col,
  2 AS new_col2,
  3 AS new_col3
FROM sushi.waiter_revenue_by_day
WHERE
  event_date = (
    SELECT
      MAX(event_date)
    FROM sushi.waiter_revenue_by_day
  )
ORDER BY
  revenue DESC
LIMIT 100